#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [2]:
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [4]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [6]:
embeddings=OllamaEmbeddings()
db=FAISS.from_documents(docs,embeddings)
db

/var/folders/z3/rlf87qn51rzbkyd7cptrtp180000gn/T/ipykernel_43873/2759370084.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings=OllamaEmbeddings()


In [8]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [10]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [13]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='94cfa0f9-807e-4710-94f9-4bfcfda5c72d', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(15611.926)),
 (Document(id='1c8b5e9f-b036-4af8-ac43-12d25ff4f98e', metadata={'source': 'spe

In [15]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[1.8803234100341797,
 -3.1001899242401123,
 1.9706624746322632,
 1.474577784538269,
 -0.9092581272125244,
 0.6635217070579529,
 1.531920075416565,
 -0.49863922595977783,
 0.9111183881759644,
 -1.9994360208511353,
 1.2914526462554932,
 -1.9512821435928345,
 -0.6407999396324158,
 1.525912880897522,
 -0.5085103511810303,
 -2.0353522300720215,
 -0.41851598024368286,
 0.06937886774539948,
 0.7285619974136353,
 -1.8505314588546753,
 -0.7135097980499268,
 -1.046565055847168,
 2.175208330154419,
 -2.005366086959839,
 0.7472622394561768,
 -0.9909680485725403,
 0.3473147749900818,
 -1.6823378801345825,
 0.17073766887187958,
 -0.865515947341919,
 2.0764126777648926,
 -2.8557865619659424,
 -2.6594676971435547,
 3.790487766265869,
 2.284578800201416,
 -5.120345592498779,
 -1.0405184030532837,
 1.3829034566879272,
 -1.2561386823654175,
 -1.3619359731674194,
 -0.4788707196712494,
 -2.028804063796997,
 2.1514439582824707,
 0.334106981754303,
 0.9196408987045288,
 0.5419221520423889,
 -0.26552656292915

In [16]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='94cfa0f9-807e-4710-94f9-4bfcfda5c72d', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='1c8b5e9f-b036-4af8-ac43-12d25ff4f98e', metadata={'source': 'speech.txt'}, page_content='To 

In [19]:
### Saving And Loading
db.save_local("faiss_index")

In [21]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [23]:
docs

[Document(id='94cfa0f9-807e-4710-94f9-4bfcfda5c72d', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='1c8b5e9f-b036-4af8-ac43-12d25ff4f98e', metadata={'source': 'speech.txt'}, page_content='To 